## Dealing with Problematic Data


### Problem in setting index in pandas DataFrame

It may happen that the dataset contains an index column. How to import it correctly with Pandas? 

We will use a very simple dataset, namely demo_df.csv (the file can be download from my github repository), that contains an index column (this is just a counter and not a feature).

In [116]:
import pandas as pd

# How to read CSV file from GitHub using pandas
# https://stackoverflow.com/questions/55240330/how-to-read-csv-file-from-github-using-pandas

# url = 'https://raw.githubusercontent.com/pairote-sat/SCMA248/main/demo_df'
url = 'https://raw.githubusercontent.com/pairote-sat/SCMA248/main/Data/demo_df.csv'

df1 = pd.read_csv(url)
print(df1.head())
df1.columns

   Unnamed: 0       dates  status
0           0  2021-03-05  Opened
1           1  2021-03-16  Opened
2           2  2021-03-28  Closed


Index(['Unnamed: 0', 'dates', 'status'], dtype='object')

In [117]:
## Uncomment these commands if the CSV dateset is stored locally.

# df1 = pd.read_csv('/Users/Kaemyuijang/SCMA248/demo_df.csv')
# print(df1.head())
# df1.columns

We want to specify that 'Unnamed: 0' is the index column  while loading this data set with the following command (with the parameter `index_col`):

In [118]:
df1 = pd.read_csv(url, index_col = 0)
df1.head()

,dates,status
0,2021-03-05,Opened
1,2021-03-16,Opened
2,2021-03-28,Closed


The dataset is loaded and the index is correct after performing the command.

### Convert Strings to Datetime

However, we see an issue right away: all of the data, including dates, has been parsed as integers (or, in other cases, as string). If the dates do not have a particularly unusual format, you can use the autodetection routines to identify the column that contains the date data. It works nicely with the following arguments when the data file is stored locally.

In [119]:
# df2 = pd.read_csv('/Users/Kaemyuijang/SCMA248/demo_df.csv', index_col = 0, parse_dates = ['dates'])
# print(df2.head())
# df2.dtypes

For the same dataset downloaded from Github, if a column or index contains an unparseable date, the entire column or index will be returned unaltered as an object data type. For non-standard datetime parsing, use pd.to_datetime after pd.read_csv:

`pd.to_datetime(df['DataFrame Column'], format=specify your format)` 

Remember that the date format for our example is yyyymmdd.

The following is a representation of this date format `format =  '%d%m%Y'` (or `format =  '%Y%m%d'`). 

See https://datatofish.com/strings-to-datetime-pandas/ for more details.

In [120]:
df2 = pd.read_csv(url, index_col = 0, parse_dates = ['dates'])
print(df2)
df2.dtypes

       dates  status
0 2021-03-05  Opened
1 2021-03-16  Opened
2 2021-03-28  Closed


dates     datetime64[ns]
status            object
dtype: object

In [121]:
df2['dates'] = pd.to_datetime(df2['dates'], format='%d%m%Y')
print(df2)
df2.dtypes

       dates  status
0 2021-03-05  Opened
1 2021-03-16  Opened
2 2021-03-28  Closed


dates     datetime64[ns]
status            object
dtype: object

### Missing values

We will concentrate on missing values, which is perhaps the most challenging data cleaning operation.

It's a good idea to have an overall sense of a data set before you start cleaning it. After that, you can develop a plan for cleaning the data.

To begin, I like to ask the following questions:

* What are the features?

* What sorts of data are required (int, float, text, boolean)?
 
* Is there any evident data missing (values that Pandas can detect)?
* Is there any other type of missing data that isn't as clear (and that Pandas can't easily detect)?

Let's have a look at an example by using a small sample data namely property_data.csv. The file can be obtained from Github: https://raw.githubusercontent.com/pairote-sat/SCMA248/main/property_data.csv.

In what follows, we also specify that 'PID' (personal indentifier) is the index column while loading this data set with the following command (with the parameter index_col):


In [122]:
url = 'https://raw.githubusercontent.com/pairote-sat/SCMA248/main/Data/property_data.csv'

df = pd.read_csv(url, index_col = 0)
df

,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
PID,,,,,,
10101.0,104.0,Khao San,Y,3,1,1000
10102.0,197.0,Silom,N,3,1.5,--
10103.0,NaN,Silom,N,NaN,1,850
10104.0,201.0,Sukhumvit,12,1,NaN,700
NaN,203.0,Sukhumvit,Y,3,2,1600
10106.0,207.0,Sukhumvit,Y,NaN,1,800
10107.0,NaN,Thonglor,NaN,2,HURLEY,950
10108.0,213.0,Rama 1,Y,--,1,NaN
10109.0,215.0,Rama 1,Y,na,2,1800


We notice that the PID (personal identifiers) as the index name has a missing value, i.e. NaN  (not any number). We will replace this missing PID with 10105 and also convert from floats to integers. 

In [123]:
rowindex = df.index.tolist()
rowindex[4] = 10105.0
rowindex = [int(i) for i in rowindex]

df.index = rowindex

print(df.loc[:,'ST_NUM'])

10101    104.0
10102    197.0
10103      NaN
10104    201.0
10105    203.0
10106    207.0
10107      NaN
10108    213.0
10109    215.0
Name: ST_NUM, dtype: float64


Alternatively, one can use Numpy to produce the same result. Simply run the following commands. Here we use `.astype()` method to convert the type of an array. 

In [124]:
df = pd.read_csv(url, index_col = 0)
df

import numpy as np
rowindex = df.index.to_numpy()

rowindex[4] = 10105.0

df.index = rowindex.astype(int)

print(df.loc[:,'ST_NUM'])

10101    104.0
10102    197.0
10103      NaN
10104    201.0
10105    203.0
10106    207.0
10107      NaN
10108    213.0
10109    215.0
Name: ST_NUM, dtype: float64


Now I can answer my first question: what are features? The following features can be obtained from the column names:

* ST_NUM is the street number

* ST_NAME is the street name

* OWN_OCCUPIED: Is the residence owner occupied?

* NUM_BEDROOMS: the number of rooms

We can also respond to the question, What are the expected types?

* ST_NUM is either a float or an int... a numeric type of some sort

* ST_NAME is a string variable.

* OWN_OCCUPIED: string; OWN_OCCUPIED: string; OWN _OCCUPIED N ("No") or Y ("Yes")

* NUM_BEDROOMS is a numeric type that can be either float or int.

### Standard Missing Values

So, what exactly do I mean when I say "standard missing values?" These are missing values that Pandas can detect.

Let's return to our initial dataset and examine the "Street Number" column.

There are an empty cell in the third row (from the original file). A value of "NaN" appears in the seventh row.

Both of these numbers are obviously missing. Let's see how Pandas handle these situations. We can see that Pandas filled in the blank space with "NaN".

We can confirm that both the missing value and "NA" were detected as missing values using the isnull() method. True for both boolean responses.

In [125]:
df['ST_NUM'].isnull()

10101    False
10102    False
10103     True
10104    False
10105    False
10106    False
10107     True
10108    False
10109    False
Name: ST_NUM, dtype: bool

Similarly, for the NUM_BEDROOMS column of the original CSV file, users manually entering missing values with different names "n/a" and "NA". Pandas also recognized these as missing values and filled with "NaN".

In [126]:
df['NUM_BEDROOMS']

10101      3
10102      3
10103    NaN
10104      1
10105      3
10106    NaN
10107      2
10108     --
10109     na
Name: NUM_BEDROOMS, dtype: object

### Missing Values That Aren't Standard

It is possible that there are missing values with different formats in some cases.

There are two other missing values in this column of different formats

* na

* `--`

Putting this different format in a list is a simple approach to detect them. When we import the data, Pandas will immediately recognize them. Here's an example of how we might go about it.

In [127]:
# Making a list of missing value types
missing_values = ["na", "--"]

df = pd.read_csv(url, index_col = 0, na_values = missing_values)

df

,ST_NUM,ST_NAME,OWN_OCCUPIED,NUM_BEDROOMS,NUM_BATH,SQ_FT
PID,,,,,,
10101.0,104.0,Khao San,Y,3.0,1,1000.0
10102.0,197.0,Silom,N,3.0,1.5,NaN
10103.0,NaN,Silom,N,NaN,1,850.0
10104.0,201.0,Sukhumvit,12,1.0,NaN,700.0
NaN,203.0,Sukhumvit,Y,3.0,2,1600.0
10106.0,207.0,Sukhumvit,Y,NaN,1,800.0
10107.0,NaN,Thonglor,NaN,2.0,HURLEY,950.0
10108.0,213.0,Rama 1,Y,NaN,1,NaN
10109.0,215.0,Rama 1,Y,NaN,2,1800.0


### Unexpected Missing Values

We have observed both standard and non-standard missing data so far. What if we have a type that is not expected?

For instance, if our feature is supposed to be a string but it's a numeric type, it's technically a missing value.

Take a look at the column labeled "OWN_OCCUPIED" to understand what I'm talking about.

In [128]:
df['OWN_OCCUPIED']

PID
10101.0      Y
10102.0      N
10103.0      N
10104.0     12
NaN          Y
10106.0      Y
10107.0    NaN
10108.0      Y
10109.0      Y
Name: OWN_OCCUPIED, dtype: object

We know Pandas will recognize the empty cell in row seven as a missing value because of our prior examples.

The number 12 appears in the fourth row. This number type should be a missing value because the result for Owner Occupied should clearly be a string (Y or N).
Because this example is a little more complicated, we will need to find a plan for identifying missing values. There are a few alternative routes to take, but this is how I'm going to go about it.

1. Loop through The OWN OCCUPIED column.

2. Convert the entry to an integer.

3. If the entry may be transformed to an integer,  enter a missing value. 

4. We know the number cannot be an integer if it cannott be an integer.

In [129]:
df = pd.read_csv(url, index_col = 0)
df

import numpy as np
rowindex = df.index.to_numpy()

rowindex[4] = 10105.0

df.index = rowindex.astype(int)


In [130]:
# Detecting numbers 
cnt=10101
for row in df['OWN_OCCUPIED']:
    try:
        int(row)
        df.loc[cnt, 'OWN_OCCUPIED']=np.nan
    except ValueError:
        pass
    cnt+=1

In [131]:
df['OWN_OCCUPIED']

10101      Y
10102      N
10103      N
10104    NaN
10105      Y
10106      Y
10107    NaN
10108      Y
10109      Y
Name: OWN_OCCUPIED, dtype: object

In the code, we loop through each entry in the "Owner Occupied" column. To try to change the entry to an integer, we use int(row). If the value can be changed to an integer, we change the entry to a missing value using np.nan from Numpy. On the other hand, if the value cannot be changed to an integer, we pass it and continue.

You will notice that I have used try and except ValueError. This is called exception handling, and we use this to handle errors.

If we tried to change an entry to an integer and it could not be changed, a ValueError would be returned and the code would terminate. To deal with this, we use exception handling to detect these errors and continue.